# 데이터셋 합치기

In [5]:
import numpy as np
import pandas as pd
import os
import shutil
import copy
from tqdm import tqdm

## 1. Train, Validation 데이터셋 합치기

### 1-1. 2022년 데이터셋 통합

In [6]:
# 현재 작업 디렉토리 가져오기
original_directory = os.getcwd()
print(f"원래 디렉토리: {original_directory}")

원래 디렉토리: c:\Users\aryij\OneDrive\문서\DataStudy\epoch\de-project


In [7]:
def combine_datasets(dir1="aihub", dir2="2022"):
    # 기본 경로 설정 (data\aihub\2022)
    base_path = os.path.join("data", dir1, dir2)

    # 2022, 2023 폴더 내부 (수도권, 동부권, 서부권, 제주도)를 region_list로 선언
    region_list = os.listdir(base_path)
    region_eng = ["capital", "east", "west", "jeju"]
    
    region_dfs = {}
    region_gps_dfs = {}


    for region, eng_region in zip(region_list, region_eng):

        print(f"{region} 데이터셋 통합 시작")

        # 지역에 따른 경로 설정
        region_path = os.path.join(base_path, region)

        if dir2=="2023":
            # 지역\3.개방데이터로 이동
            mid_path = os.path.join(region_path, os.listdir(region_path)[0]) 

            # 첫 번째 폴더 내부의 첫 번째 하위 폴더로 이동 (지역\3.개방데이터\1.데이터)
            data_path = os.path.join(mid_path, os.listdir(mid_path)[0])
        
        else:
            # 지역\01-1.정식개방데이터로 이동
            data_path = os.path.join(region_path, os.listdir(region_path)[0])  

        # train, validation 데이터셋 combine하기 위해 경로 설정 및 폴더 생성
        combine_path = os.path.join(data_path, "combined")

        # 기존 폴더가 있다면 삭제
        if os.path.exists(combine_path):
            shutil.rmtree(combine_path)

        os.makedirs(combine_path, exist_ok=True)

        # 01-1.정식개방데이터 내부 폴더 리스트 가져온다
        train_val_list = os.listdir(data_path)

        # data_dict, gps_dict = {}, {}

        region_dfs[eng_region] = {}
        region_gps_dfs[eng_region] = {}

        for folder in train_val_list:
            if (folder == "combined") or (folder == "Other") or (folder == "concatenated"):
                continue
            
            # 02.라벨링데이터 폴더 내부로 이동
            folder_path = os.path.join(data_path, folder, "02.라벨링데이터")

            # .zip (압축파일)이 경로에 있는 경우 리스트에서 뺀다
            labeling_folders = [f for f in os.listdir(folder_path) if not f.endswith(".zip")]

            for subfolder in labeling_folders:
                # 02.라벨링데이터 내부 경로 가져온다
                subfolder_path = os.path.join(folder_path, subfolder)

                # csv_list가 메타데이터라면 불러오지 않는다
                csv_files = [f for f in os.listdir(subfolder_path) if not "poi" in f]

                for csv_file in csv_files:
                    # csv 파일 경로 불러오기
                    file_path = os.path.join(subfolder_path, csv_file)
                    # .csv를 빈 칸으로 replace
                    df_name = csv_file.replace(".csv", "")
                    
                    # 이름 중복 검사 및 수정 (data_dict와 gps_dict 모두 확인)
                    if df_name in region_dfs[eng_region].keys() or df_name in region_gps_dfs.keys():
                        df_name = f"{df_name}_val"  # 중복 시 '이름 뒤에 _val' 추가

                    df = pd.read_csv(file_path)
                    
                    # gps 데이터인 경우 바로 combine_path에 저장
                    if "gps" in df_name.lower():
                        region_gps_dfs[eng_region][df_name] = df

                    else:
                        region_dfs[eng_region][df_name] = df
                        # region_dfs = make_region_column(region_dfs)

        # region_dfs[eng_region] 파일의 key를 순서대로 정렬한 후 list로 변경
        key_list = sorted(region_dfs[eng_region].keys())
        print(f"{region} 데이터: {key_list}\n")
        key_pairs = [(key_list[i], key_list[i + 1]) for i in range(0, len(key_list), 2)]

        for key_train, key_val in key_pairs:
            # 데이터프레임 세로로 병합
            # combined_df = pd.combine([region_dfs[eng_region][key_train], region_dfs[eng_region][key_val]], axis=0)
            all_data = []
            all_data.extend(region_dfs[eng_region][key_train].to_dict(orient="records"))
            all_data.extend(region_dfs[eng_region][key_val].to_dict(orient="records"))

            # combine 결과를 저장
            region_dfs[eng_region][f"{key_train}_combine"] = pd.DataFrame(all_data)
            
            # print(f"{key_train} Train 데이터셋 사이즈:      {region_dfs[eng_region][key_train].shape[0]}")
            # print(f"{key_train} Validation 데이터셋 사이즈: {region_dfs[eng_region][key_val].shape[0]}")
            # print(f"{key_train} Combined 데이터셋 사이즈:   {pd.DataFrame(all_data).shape[0]}")
            # print("------------------------------------------------------------------------------\n")

        print(f"========================== {region} 데이터 통합 완료 ==========================\n")

    return region_dfs, region_gps_dfs

In [8]:
# 2022년 데이터셋 combine
region_dfs_22, region_gps_dfs_22 = combine_datasets("aihub", "2022")

277.국내 여행로그 데이터(수도권) 데이터셋 통합 시작
277.국내 여행로그 데이터(수도권) 데이터: ['tc_codea_코드A', 'tc_codea_코드A_val', 'tc_codeb_코드B', 'tc_codeb_코드B_val', 'tc_sgg_시군구코드', 'tc_sgg_시군구코드_val', 'tn_activity_consume_his_활동소비내역_A', 'tn_activity_consume_his_활동소비내역_A_val', 'tn_activity_his_활동내역_A', 'tn_activity_his_활동내역_A_val', 'tn_adv_consume_his_사전소비내역_A', 'tn_adv_consume_his_사전소비내역_A_val', 'tn_companion_info_동반자정보_A', 'tn_companion_info_동반자정보_A_val', 'tn_lodge_consume_his_숙박소비내역_A', 'tn_lodge_consume_his_숙박소비내역_A_val', 'tn_move_his_이동내역_A', 'tn_move_his_이동내역_A_val', 'tn_mvmn_consume_his_이동수단소비내역_A', 'tn_mvmn_consume_his_이동수단소비내역_A_val', 'tn_tour_photo_관광사진_A', 'tn_tour_photo_관광사진_A_val', 'tn_travel_여행_A', 'tn_travel_여행_A_val', 'tn_traveller_master_여행객 Master_A', 'tn_traveller_master_여행객 Master_A_val', 'tn_visit_area_info_방문지정보_A', 'tn_visit_area_info_방문지정보_A_val']

========================== 277.국내 여행로그 데이터(수도권) 데이터 통합 완료 ==========================

278.국내 여행로그 데이터(동부권) 데이터셋 통합 시작


C:\Users\aryij\AppData\Local\Temp\ipykernel_13152\3611837494.py:75: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


278.국내 여행로그 데이터(동부권) 데이터: ['tc_codea_코드A', 'tc_codea_코드A_val', 'tc_codeb_코드B', 'tc_codeb_코드B_val', 'tc_sgg_시군구코드', 'tc_sgg_시군구코드_val', 'tn_activity_consume_his_활동소비내역_B', 'tn_activity_consume_his_활동소비내역_B_val', 'tn_activity_his_활동내역_B', 'tn_activity_his_활동내역_B_val', 'tn_adv_consume_his_사전소비내역_B', 'tn_adv_consume_his_사전소비내역_B_val', 'tn_companion_info_동반자정보_B', 'tn_companion_info_동반자정보_B_val', 'tn_lodge_consume_his_숙박소비내역_B', 'tn_lodge_consume_his_숙박소비내역_B_val', 'tn_move_his_이동내역_B', 'tn_move_his_이동내역_B_val', 'tn_mvmn_consume_his_이동수단소비내역_B', 'tn_mvmn_consume_his_이동수단소비내역_B_val', 'tn_tour_photo_관광사진_B', 'tn_tour_photo_관광사진_B_val', 'tn_travel_여행_B', 'tn_travel_여행_B_val', 'tn_traveller_master_여행객 Master_B', 'tn_traveller_master_여행객 Master_B_val', 'tn_visit_area_info_방문지정보_B', 'tn_visit_area_info_방문지정보_B_val']

========================== 278.국내 여행로그 데이터(동부권) 데이터 통합 완료 ==========================

279.국내 여행로그 데이터(서부권) 데이터셋 통합 시작


C:\Users\aryij\AppData\Local\Temp\ipykernel_13152\3611837494.py:75: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


279.국내 여행로그 데이터(서부권) 데이터: ['tc_codea_코드A', 'tc_codea_코드A_val', 'tc_codeb_코드B', 'tc_codeb_코드B_val', 'tc_sgg_시군구코드', 'tc_sgg_시군구코드_val', 'tn_activity_consume_his_활동소비내역_C', 'tn_activity_consume_his_활동소비내역_C_val', 'tn_activity_his_활동내역_C', 'tn_activity_his_활동내역_C_val', 'tn_adv_consume_his_사전소비내역_C', 'tn_adv_consume_his_사전소비내역_C_val', 'tn_companion_info_동반자정보_C', 'tn_companion_info_동반자정보_C_val', 'tn_lodge_consume_his_숙박소비내역_C', 'tn_lodge_consume_his_숙박소비내역_C_val', 'tn_move_his_이동내역_C', 'tn_move_his_이동내역_C_val', 'tn_mvmn_consume_his_이동수단소비내역_C', 'tn_mvmn_consume_his_이동수단소비내역_C_val', 'tn_tour_photo_관광사진_C', 'tn_tour_photo_관광사진_C_val', 'tn_travel_여행_C', 'tn_travel_여행_C_val', 'tn_traveller_master_여행객 Master_C', 'tn_traveller_master_여행객 Master_C_val', 'tn_visit_area_info_방문지정보_C', 'tn_visit_area_info_방문지정보_C_val']

========================== 279.국내 여행로그 데이터(서부권) 데이터 통합 완료 ==========================

280.국내 여행로그 데이터(제주도 및 도서지역) 데이터셋 통합 시작


C:\Users\aryij\AppData\Local\Temp\ipykernel_13152\3611837494.py:75: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\aryij\AppData\Local\Temp\ipykernel_13152\3611837494.py:75: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


280.국내 여행로그 데이터(제주도 및 도서지역) 데이터: ['tc_codea_코드A', 'tc_codea_코드A_val', 'tc_codeb_코드B', 'tc_codeb_코드B_val', 'tc_sgg_시군구코드', 'tc_sgg_시군구코드_val', 'tn_activity_consume_his_활동소비내역_D', 'tn_activity_consume_his_활동소비내역_D_val', 'tn_activity_his_활동내역_D', 'tn_activity_his_활동내역_D_val', 'tn_adv_consume_his_사전소비내역_D', 'tn_adv_consume_his_사전소비내역_D_val', 'tn_companion_info_동반자정보_D', 'tn_companion_info_동반자정보_D_val', 'tn_lodge_consume_his_숙박소비내역_D', 'tn_lodge_consume_his_숙박소비내역_D_val', 'tn_move_his_이동내역_D', 'tn_move_his_이동내역_D_val', 'tn_mvmn_consume_his_이동수단소비내역_D', 'tn_mvmn_consume_his_이동수단소비내역_D_val', 'tn_tour_photo_관광사진_D', 'tn_tour_photo_관광사진_D_val', 'tn_travel_여행_D', 'tn_travel_여행_D_val', 'tn_traveller_master_여행객 Master_D', 'tn_traveller_master_여행객 Master_D_val', 'tn_visit_area_info_방문지정보_D', 'tn_visit_area_info_방문지정보_D_val']

========================== 280.국내 여행로그 데이터(제주도 및 도서지역) 데이터 통합 완료 ==========================



### 1-2. 2023년 데이터셋 통합

In [9]:
# 2023년 데이터셋 combine
region_dfs_23, region_gps_dfs_23 = combine_datasets("aihub", "2023")

145.국내 여행로그 데이터_수도권_2차년도 데이터셋 통합 시작
145.국내 여행로그 데이터_수도권_2차년도 데이터: ['tc_codea_코드A', 'tc_codea_코드A_val', 'tc_codeb_코드B', 'tc_codeb_코드B_val', 'tc_sgg_시군구코드', 'tc_sgg_시군구코드_val', 'tn_activity_consume_his_활동소비내역_E', 'tn_activity_consume_his_활동소비내역_E_val', 'tn_activity_his_활동내역_E', 'tn_activity_his_활동내역_E_val', 'tn_adv_consume_his_사전소비내역_E', 'tn_adv_consume_his_사전소비내역_E_val', 'tn_companion_info_동반자정보_E', 'tn_companion_info_동반자정보_E_val', 'tn_lodge_consume_his_숙박소비내역_E', 'tn_lodge_consume_his_숙박소비내역_E_val', 'tn_move_his_이동내역_E', 'tn_move_his_이동내역_E_val', 'tn_mvmn_consume_his_이동수단소비내역_E', 'tn_mvmn_consume_his_이동수단소비내역_E_val', 'tn_tour_photo_관광사진_E', 'tn_tour_photo_관광사진_E_val', 'tn_travel_여행_E', 'tn_travel_여행_E_val', 'tn_traveller_master_여행객 Master_E', 'tn_traveller_master_여행객 Master_E_val', 'tn_visit_area_info_방문지정보_E', 'tn_visit_area_info_방문지정보_E_val']

========================== 145.국내 여행로그 데이터_수도권_2차년도 데이터 통합 완료 ==========================

146.국내 여행로그 데이터_동부권_2차년도 데이터셋 통합 시작
146.국내 여행로그 데이터_

In [10]:
# size = 0
# for df in region_gps_dfs_23["capital"].keys():
#     size += region_gps_dfs_23["capital"][df].shape[0]
# size

In [11]:
# 2022년 gps 데이터 capital train + validation 3600개 동일
len(region_gps_dfs_22["capital"].keys())

3600

In [12]:
# 2023년 gps 데이터 capital train + validation 2880개 동일
len(region_gps_dfs_23["capital"].keys())

2880

In [13]:
# 2022년 gps 데이터 jeju train + validation 3600개 동일
len(region_gps_dfs_22["jeju"].keys())

3600

In [14]:
# 2023년 gps 데이터 jeju train + validation 2880개 동일
len(region_gps_dfs_23["jeju"].keys())

2880

In [15]:
# 중복되는 데이터 확인 → 로그 데이터의 DT_MIN이 달라도 중복데이터로 취급
# 로그데이터는 활동내역, 방문지, 소비내역 등 기록
# 같은 시간에 같은 장소에서 여러번 발생할 수 있다 → 중복 제거하지 않는다
region_gps_dfs_22["capital"]["tn_gps_coord_a_a002571"][region_gps_dfs_22["capital"]["tn_gps_coord_a_a002571"].duplicated()]

,MOBILE_NUM_ID,X_COORD,Y_COORD,DT_MIN,TRAVEL_ID
1,96ffd6494524add7,126.711703,37.459781,2022-10-02 08:16,a_a002571
3,96ffd6494524add7,126.711703,37.459781,2022-10-02 08:17,a_a002571
5,96ffd6494524add7,126.711703,37.459781,2022-10-02 08:18,a_a002571
9,96ffd6494524add7,126.711467,37.460012,2022-10-02 08:21,a_a002571
10,96ffd6494524add7,126.711467,37.460012,2022-10-02 08:21,a_a002571
...,...,...,...,...,...
1351,96ffd6494524add7,126.711689,37.459738,2022-10-02 20:00,a_a002571
1352,96ffd6494524add7,126.711689,37.459738,2022-10-02 20:00,a_a002571
1354,96ffd6494524add7,126.711689,37.459738,2022-10-02 20:01,a_a002571
1356,96ffd6494524add7,126.711689,37.459738,2022-10-02 20:02,a_a002571


## 2. 데이터셋에 region 컬럼 생성

In [16]:
# 통합한 데이터셋에 region 컬럼 생성하는 함수
def make_region_column(region_dict):

    # 딕셔너리 key를 받아온다
    region_keys = region_dict.keys()
    
    # region_keys: capital, east, west, jeju
    for region_key in region_keys:
        
        # region_key를 바탕으로, 각각의 지역 필터링한 후 keys를 다시 찾는다 (데이터셋 이름들)
        dataset_keys = region_dict[region_key].keys()

        for dataset_key in dataset_keys:
            # 지역별 데이터셋들을 각각 불러와서 region 컬럼을 생성하고 해당 지역을 값에 넣는다
            regional_data = region_dict[region_key][dataset_key]
            regional_data["region"] = f"{region_key}"
    
    return region_dict

In [17]:
# 22년, 23년 데이터에 make_region_column 함수 적용
region_dfs_22 = make_region_column(region_dfs_22)
region_gps_dfs_22 = make_region_column(region_gps_dfs_22)

region_dfs_23 = make_region_column(region_dfs_23)
region_gps_dfs_23 = make_region_column(region_gps_dfs_23)

In [18]:
# region_gps_dfs_22 확인
region_gps_dfs_22["capital"]["tn_gps_coord_a_a000011"].head(3)

,MOBILE_NUM_ID,X_COORD,Y_COORD,DT_MIN,TRAVEL_ID,region
0,a9c1a065c89da3b7,126.702718,37.500779,2022-08-06 12:33,a_a000011,capital
1,a9c1a065c89da3b7,126.703540,37.506400,2022-08-06 12:34,a_a000011,capital
2,a9c1a065c89da3b7,126.706564,37.506687,2022-08-06 12:35,a_a000011,capital


In [19]:
# region_gps_dfs_23 확인
region_gps_dfs_23["east"]["tn_gps_coord_e_e000003"].head(3)

,MOBILE_NUM_ID,X_COORD,Y_COORD,DT_MIN,TRAVEL_ID,region
0,e000003_900008,126.702967,37.628667,2023-04-30 07:19,e_e000003,east
1,e000003_900008,126.702967,37.628667,2023-04-30 07:20,e_e000003,east
2,e000003_900008,126.702967,37.628667,2023-04-30 07:21,e_e000003,east


In [20]:
# 2022년 데이터셋 train, validation combine 제대로 되었는지 확인

print("tn_activity_consume_his_활동소비내역_D Train 데이터:     ", region_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D"].shape)
print("tn_activity_consume_his_활동소비내역_D Validation 데이터: ", region_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D_val"].shape)
print("tn_activity_consume_his_활동소비내역_D Combine 데이터:    ", region_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D_combine"].shape)

if region_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D"].shape[0] + region_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D_val"].shape[0] == region_dfs_22["jeju"]["tn_activity_consume_his_활동소비내역_D_combine"].shape[0]:
    print("\n데이터 combine이 제대로 되었습니다!")
else:
    print("\n데이터 combine에 문제가 생겼습니다.")

tn_activity_consume_his_활동소비내역_D Train 데이터:      (33796, 19)
tn_activity_consume_his_활동소비내역_D Validation 데이터:  (4224, 19)
tn_activity_consume_his_활동소비내역_D Combine 데이터:     (38020, 19)

데이터 combine이 제대로 되었습니다!


In [21]:
# 2023년 데이터셋 train, validation combine 제대로 되었는지 확인

print("tn_activity_consume_his_활동소비내역_H Train 데이터:     ", region_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H"].shape)
print("tn_activity_consume_his_활동소비내역_H Validation 데이터: ", region_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H_val"].shape)
print("tn_activity_consume_his_활동소비내역_H Combine 데이터:    ", region_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H_combine"].shape)

if region_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H"].shape[0] + region_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H_val"].shape[0] == region_dfs_23["jeju"]["tn_activity_consume_his_활동소비내역_H_combine"].shape[0]:
    print("\n데이터 combine이 제대로 되었습니다!")
else:
    print("\n데이터 combine에 문제가 생겼습니다.")

tn_activity_consume_his_활동소비내역_H Train 데이터:      (26942, 19)
tn_activity_consume_his_활동소비내역_H Validation 데이터:  (3184, 19)
tn_activity_consume_his_활동소비내역_H Combine 데이터:     (30126, 19)

데이터 combine이 제대로 되었습니다!


## 3. 데이터프레임 이름 변경
- train, validation 데이터를 combine한 데이터프레임의 이름이 `column_combine` 형식으로 되어있다
- `_combine` 글자를 제거한다

In [22]:
# train, validation combine한 데이터프레임 이름 변경 함수 (_combine 글자 제거)
def update_keys(data_dict, new_keys):

    # data_dict 딕셔너리의 딥 카피 생성 (원본은 보존하기 위함)
    data_dict_copy = copy.deepcopy(data_dict)
    
    for region, datasets in data_dict_copy.items():
        # "combine" 포함된 key만 필터링
        keys_to_update = [key for key in datasets.keys() if "combine" in key]
        
        # 새 키로 변경
        for idx, key in enumerate(keys_to_update):
            if idx < len(new_keys):  # new_keys 범위 내에서 변경
                new_key = new_keys[idx]
                datasets[new_key] = datasets.pop(key)
                # print(f"{region} {datasets} 크기:      {datasets[new_key].shape[0]}".rjust(5))

        # "combine"로 변경된 key만 남기도록 필터링
        keys_to_remove = [key for key in datasets.keys() if key not in new_keys]
        for key in keys_to_remove:
            datasets.pop(key)
    
    return data_dict_copy

In [23]:
# _combine으로 되어있는 key 값들에서 combine 제외
keys_combine = ['tc_codea_코드A',
               'tc_codeb_코드B',
               'tc_sgg_시군구코드',
               'tn_activity_consume_his_활동소비내역',
               'tn_activity_his_활동내역',
               'tn_adv_consume_his_사전소비내역',
               'tn_companion_info_동반자정보',
               'tn_lodge_consume_his_숙박소비내역',
               'tn_move_his_이동내역',
               'tn_mvmn_consume_his_이동수단소비내역',
               'tn_tour_photo_관광사진',
               'tn_travel_여행',
               'tn_traveller_master_여행객 Master',
               'tn_visit_area_info_방문지정보']

updated_dfs_22 = update_keys(region_dfs_22, keys_combine)

In [24]:
updated_dfs_23 = update_keys(region_dfs_23, keys_combine)

In [25]:
# 필터링되고, 바뀐 key 값들 확인
for region, types in updated_dfs_22.items():
    print(f"updated_dfs_22 {region}: {list(types.keys())}")

print("")

# 필터링되고, 바뀐 key 값들 확인
for region, types in updated_dfs_23.items():
    print(f"updated_dfs_23 {region}: {list(types.keys())}")

updated_dfs_22 capital: ['tc_codea_코드A', 'tc_codeb_코드B', 'tc_sgg_시군구코드', 'tn_activity_consume_his_활동소비내역', 'tn_activity_his_활동내역', 'tn_adv_consume_his_사전소비내역', 'tn_companion_info_동반자정보', 'tn_lodge_consume_his_숙박소비내역', 'tn_move_his_이동내역', 'tn_mvmn_consume_his_이동수단소비내역', 'tn_tour_photo_관광사진', 'tn_travel_여행', 'tn_traveller_master_여행객 Master', 'tn_visit_area_info_방문지정보']
updated_dfs_22 east: ['tc_codea_코드A', 'tc_codeb_코드B', 'tc_sgg_시군구코드', 'tn_activity_consume_his_활동소비내역', 'tn_activity_his_활동내역', 'tn_adv_consume_his_사전소비내역', 'tn_companion_info_동반자정보', 'tn_lodge_consume_his_숙박소비내역', 'tn_move_his_이동내역', 'tn_mvmn_consume_his_이동수단소비내역', 'tn_tour_photo_관광사진', 'tn_travel_여행', 'tn_traveller_master_여행객 Master', 'tn_visit_area_info_방문지정보']
updated_dfs_22 west: ['tc_codea_코드A', 'tc_codeb_코드B', 'tc_sgg_시군구코드', 'tn_activity_consume_his_활동소비내역', 'tn_activity_his_활동내역', 'tn_adv_consume_his_사전소비내역', 'tn_companion_info_동반자정보', 'tn_lodge_consume_his_숙박소비내역', 'tn_move_his_이동내역', 'tn_mvmn_consume_his_이동수단소비내

In [26]:
# 데이터 숫자 누락 확인 → 누락 없음
print(updated_dfs_22["capital"]["tn_move_his_이동내역"].shape[0])
print(updated_dfs_22["east"]["tn_activity_consume_his_활동소비내역"].shape[0])

33913
22302


## 4. Region 통합
- 현재 `data_dict`은 `data_dict["region"]["csv_name"]`의 구조로 되어있다 (Nested Dictionary)
- `region`을 통합해준다

In [27]:
def combine_dict_total_region(data_dict, is_gps="no"):
    combine_total_region = {}

    # 이중 딕셔너리 순회
    for region, datasets in tqdm(data_dict.items()):
        for df_name, df in datasets.items():
            if df_name not in combine_total_region:
                combine_total_region[df_name] = []  # type별로 리스트 초기화
            
            # 각 데이터프레임을 딕셔너리 레코드로 변환 후 리스트에 추가
            combine_total_region[df_name].extend(df.to_dict(orient='records'))

    # 모든 데이터를 DataFrame으로 변환
    # gps 데이터가 아니면 병합 후 중복 제거
    if is_gps.lower() == "no":
        combine_total_region = {
            key: pd.DataFrame(records).drop_duplicates() for key, records in combine_total_region.items()
        }
    # gps 데이터라면 병합 후 중복을 제거하지 않는다
    else:
        combine_total_region = {
            key: pd.DataFrame(records) for key, records in combine_total_region.items()
        }

    return combine_total_region


In [ ]:
# updated_dfs_22 -> capital -> 시군구코드
# updated_dfs_22 -> 시군구코드

In [30]:
# # 딕셔너리 파일이 너무 커서 1/2로 줄인다
# def divide_dictionary(dictionary, length):
#     divide_length = len(dictionary) // length
#     items = list(dictionary.items())

#     # 슬라이싱
#     first_divide = dict(items[:divide_length])
#     second_divide = dict(items[divide_length:])

#     return first_divide, second_divide

def divide_dictionary(dictionary, length):
    divide_length = len(dictionary) // length  # length가 4라면 1/4로 나눔
    items = list(dictionary.items())

    # 슬라이싱
    first_divide = dict(items[:divide_length])  # 첫 번째 1/4
    second_divide = dict(items[divide_length:2*divide_length])  # 두 번째 1/4
    third_divide = dict(items[2*divide_length:3*divide_length])  # 세 번째 1/4
    fourth_divide = dict(items[3*divide_length:])  # 나머지 (네 번째 1/4)

    return first_divide, second_divide, third_divide, fourth_divide

In [31]:
# region_gps_dfs_22_first, region_gps_dfs_22_second = divide_dictionary(region_gps_dfs_22, 2)
# region_gps_dfs_23_first, region_gps_dfs_23_second = divide_dictionary(region_gps_dfs_23, 2)

region_gps_dfs_22_first, region_gps_dfs_22_second, region_gps_dfs_22_third, region_gps_dfs_22_fourth = divide_dictionary(region_gps_dfs_22, 4)
region_gps_dfs_23_first, region_gps_dfs_23_second, region_gps_dfs_23_third, region_gps_dfs_23_fourth = divide_dictionary(region_gps_dfs_23, 4)

In [32]:
# 지역 단위로 나뉘어져 있던 train, validation combine한 데이터프레임을 통합해준다
total_dfs_22 = combine_dict_total_region(updated_dfs_22)

100%|██████████| 4/4 [00:09<00:00,  2.43s/it]


In [33]:
total_dfs_23 = combine_dict_total_region(updated_dfs_23)

100%|██████████| 4/4 [00:04<00:00,  1.09s/it]


In [34]:
# 지역 단위로 나뉘어져 있던 gps 데이터프레임을 통합해준다
# total_gps_dfs_22 = combine_dict_total_region(region_gps_dfs_22, is_gps="yes")
total_gps_dfs_22_first = combine_dict_total_region(region_gps_dfs_22_first, is_gps="yes")

100%|██████████| 1/1 [00:36<00:00, 36.82s/it]


In [35]:
total_gps_dfs_22_second = combine_dict_total_region(region_gps_dfs_22_second, is_gps="yes")

100%|██████████| 1/1 [00:54<00:00, 54.87s/it]


In [36]:
total_gps_dfs_22_third = combine_dict_total_region(region_gps_dfs_22_third, is_gps="yes")

100%|██████████| 1/1 [01:01<00:00, 61.99s/it]


In [37]:
total_gps_dfs_22_fourth = combine_dict_total_region(region_gps_dfs_22_fourth, is_gps="yes")

100%|██████████| 1/1 [01:34<00:00, 94.43s/it]


In [38]:
# total_gps_dfs_23 = combine_dict_total_region(region_gps_dfs_23, is_gps="yes")
total_gps_dfs_23_first = combine_dict_total_region(region_gps_dfs_23_first, is_gps="yes")

100%|██████████| 1/1 [01:01<00:00, 61.59s/it]


In [39]:
total_gps_dfs_23_second = combine_dict_total_region(region_gps_dfs_23_second, is_gps="yes")

100%|██████████| 1/1 [01:54<00:00, 114.51s/it]


In [40]:
total_gps_dfs_23_third = combine_dict_total_region(region_gps_dfs_23_third, is_gps="yes")

100%|██████████| 1/1 [01:41<00:00, 101.61s/it]


In [41]:
total_gps_dfs_23_fourth = combine_dict_total_region(region_gps_dfs_23_fourth, is_gps="yes")

100%|██████████| 1/1 [02:44<00:00, 164.54s/it]


## 5. 최종적으로 통합한 데이터 저장

### 5-1. data_dict (gps 데이터, csv 데이터) 저장

In [42]:
# 지역별 데이터 통합, 저장 함수 생성
def store_total_region_combine(data_dict, dir1="aihub", dir2="2022", is_gps="no"):
    # 기본 경로 설정 (data\aihub\2022)
    base_path = os.path.join("data", dir1, dir2)

    # gps 데이터가 아니라면
    if is_gps == "no":
        combine_path = os.path.join(base_path, "total_combined", "region_data")
    else:
        combine_path = os.path.join(base_path, "total_combined", "gps_data")

    # 기존 폴더가 있다면 삭제
    if os.path.exists(combine_path):
        shutil.rmtree(combine_path)

    # total_combined 폴더 생성 (2022\total_combined)
    os.makedirs(combine_path, exist_ok=True)

    for key in data_dict.keys():
        print(f"{key} 데이터셋 저장 시작 \n저장 경로: {combine_path}\n")
        data_dict[key].to_csv(os.path.join(combine_path, f"{key}.csv"), index=False)
    
    print(f"========================== {dir2} 통합 데이터 저장 완료 ==========================\n")

In [43]:
store_total_region_combine(total_dfs_22, dir1="aihub", dir2="2022", is_gps="no")

tc_codea_코드A 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tc_codeb_코드B 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tc_sgg_시군구코드 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_activity_consume_his_활동소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_activity_his_활동내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_adv_consume_his_사전소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_companion_info_동반자정보 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_lodge_consume_his_숙박소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_move_his_이동내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_mvmn_consume_his_이동수단소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_tour_photo_관광사진 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_data

tn_travel_여행 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\region_dat

In [44]:
store_total_region_combine(total_dfs_23, dir1="aihub", dir2="2023", is_gps="no")

tc_codea_코드A 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tc_codeb_코드B 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tc_sgg_시군구코드 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_activity_consume_his_활동소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_activity_his_활동내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_adv_consume_his_사전소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_companion_info_동반자정보 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_lodge_consume_his_숙박소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_move_his_이동내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_mvmn_consume_his_이동수단소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_tour_photo_관광사진 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_data

tn_travel_여행 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\region_dat

In [45]:
store_total_region_combine(total_gps_dfs_22_first, dir1="aihub", dir2="2022", is_gps="yes")

tn_gps_coord_a_a000011 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000012 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000013 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000016 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000018 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000019 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000021 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000028 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000029 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000030 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000034 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000038 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_g

In [46]:
store_total_region_combine(total_gps_dfs_22_second, dir1="aihub", dir2="2022", is_gps="yes")

tn_gps_coord_a_a000316 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000330 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000594 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a001011 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a001597 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a001630 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a001698 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a001928 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a002265 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a002311 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a003054 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a003500 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_g

In [47]:
store_total_region_combine(total_gps_dfs_22_third, dir1="aihub", dir2="2022", is_gps="yes")

tn_gps_coord_a_a000273 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000319 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000777 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a001091 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a001146 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a001707 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a001711 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a001849 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a001916 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a002255 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a002375 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a002569 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_g

In [48]:
store_total_region_combine(total_gps_dfs_22_fourth, dir1="aihub", dir2="2022", is_gps="yes")

tn_gps_coord_a_a000047 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000159 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000169 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000205 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000220 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000275 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000310 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000311 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000317 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000321 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000343 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_gps_coord_a_a000353 데이터셋 저장 시작 
저장 경로: data\aihub\2022\total_combined\gps_data

tn_g

In [49]:
store_total_region_combine(total_gps_dfs_23_first, dir1="aihub", dir2="2023", is_gps="yes")

tn_gps_coord_e_e000004 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000006 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000009 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000010 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000011 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000013 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000014 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000015 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000016 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000018 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000020 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000021 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_g

In [50]:
store_total_region_combine(total_gps_dfs_23_second, dir1="aihub", dir2="2023", is_gps="yes")

tn_gps_coord_e_e000003 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000340 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000515 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000571 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000601 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000654 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e001062 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e004690 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e004785 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e006945 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e008914 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_f_f000007 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_g

In [51]:
store_total_region_combine(total_gps_dfs_23_third, dir1="aihub", dir2="2023", is_gps="yes")

tn_gps_coord_e_e000005 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000043 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000191 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000316 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000668 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e001033 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e001240 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e001423 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e001579 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e002195 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e003226 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e004870 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_g

In [52]:
store_total_region_combine(total_gps_dfs_23_fourth, dir1="aihub", dir2="2023", is_gps="yes")

tn_gps_coord_e_e000295 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000305 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000335 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000365 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000401 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000437 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000443 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000463 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000506 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000559 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000574 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_gps_coord_e_e000580 데이터셋 저장 시작 
저장 경로: data\aihub\2023\total_combined\gps_data

tn_g

### 5-2. 메타데이터 파일 복사
- `data\aihub\metadata` 경로로 파일 복사

In [53]:
# 메타데이터 파일 이동
metadata = "aihub\\2023\\145.국내 여행로그 데이터_수도권_2차년도\\3.개방데이터\\1.데이터\\Other\\Other\\메타데이터\\tn_poi_master_POIMaster_all.csv"
source_path = os.path.join("data", metadata)
# print(source_path)

# 대상 폴더 경로
meta_path = os.path.join("data", "aihub", "metadata")
os.makedirs(meta_path, exist_ok=True)

# 파일 복사
shutil.copy(source_path, meta_path)

print(f"메타데이터 파일이 {meta_path}로 성공적으로 복사되었습니다!")

메타데이터 파일이 data\aihub\metadata로 성공적으로 복사되었습니다!
